In [454]:
import pandas as pd
from datetime import date

In [455]:
import matplotlib.pyplot as plt
import seaborn as sns

In [456]:
animals = pd.read_csv("project1.csv")
animals.head()

,Animal ID,Date of Birth,Name,DateTime,MonthYear,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A680855,5/25/14,NaN,2014-06-10T00:00:00-05:00,Jun-14,Transfer,Partner,Bird,Unknown,2 weeks,Duck,Yellow/Black
1,A680857,5/25/14,NaN,2014-06-10T00:00:00-05:00,Jun-14,Transfer,Partner,Bird,Unknown,2 weeks,Duck,Yellow/Black
2,A680858,5/25/14,NaN,2014-06-10T00:00:00-05:00,Jun-14,Transfer,Partner,Bird,Unknown,2 weeks,Duck,Yellow/Black
3,A680859,5/25/14,NaN,2014-06-10T00:00:00-05:00,Jun-14,Transfer,Partner,Bird,Unknown,2 weeks,Duck,Yellow/Black
4,A680860,5/25/14,NaN,2014-06-10T00:00:00-05:00,Jun-14,Transfer,Partner,Bird,Unknown,2 weeks,Duck,Yellow/Black


In [457]:
animals.shape

(131165, 12)

In [458]:
animals.size

1573980

In [459]:
animals.info()
animals.duplicated().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131165 entries, 0 to 131164
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Animal ID         131165 non-null  object
 1   Date of Birth     131165 non-null  object
 2   Name              93658 non-null   object
 3   DateTime          131165 non-null  object
 4   MonthYear         131165 non-null  object
 5   Outcome Type      131125 non-null  object
 6   Outcome Subtype   65810 non-null   object
 7   Animal Type       131165 non-null  object
 8   Sex upon Outcome  131165 non-null  object
 9   Age upon Outcome  131165 non-null  object
 10  Breed             131165 non-null  object
 11  Color             131165 non-null  object
dtypes: object(12)
memory usage: 12.0+ MB


17

In [460]:
animals.isnull().sum()

Animal ID               0
Date of Birth           0
Name                37507
DateTime                0
MonthYear               0
Outcome Type           40
Outcome Subtype     65355
Animal Type             0
Sex upon Outcome        0
Age upon Outcome        0
Breed                   0
Color                   0
dtype: int64

In [461]:
# All of the data types are objects and likely need to be converted to numerical values for valuable ML results. 
# There are also 17 duplicates among the rows and thus need to be treated. 
# I would convert Date of Birth, DateTime, MonthYear by first converting to datetime. Date of Birth can be converted to age.
# Animal ID and Name don't really need to be changed and can be eliminated as all IDs are unique and Names don't help analysis.
# Outcome Type, Outcome Subtype, Animal Type, Sex upon Outcome, Breed, and Color can be converted to categorial using One-hot encoding. 
# Lastly, Age Upon Outcome can be stripped to only a numerical value of days so everything becomes cohesive. 
animals.drop_duplicates(inplace=True, ignore_index=True)
animals.duplicated().sum()
# Lastly the Animal ID column, the Name column should be eliminated as they do not add any value to the analysis as each value are unique or don't affect the analysis numerically.
animals.drop(["Animal ID"], axis = 1, inplace=True)
animals.drop(["Name"], axis = 1, inplace=True)

In [462]:
animals.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131148 entries, 0 to 131147
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Date of Birth     131148 non-null  object
 1   DateTime          131148 non-null  object
 2   MonthYear         131148 non-null  object
 3   Outcome Type      131108 non-null  object
 4   Outcome Subtype   65802 non-null   object
 5   Animal Type       131148 non-null  object
 6   Sex upon Outcome  131148 non-null  object
 7   Age upon Outcome  131148 non-null  object
 8   Breed             131148 non-null  object
 9   Color             131148 non-null  object
dtypes: object(10)
memory usage: 10.0+ MB


In [463]:
# Now after removing duplicated rows and useless columns the data types of each column should be converted to useful values.
# Convert DOB to datetime then solve for the age given today's date.
animals["Date of Birth"] = pd.to_datetime(animals["Date of Birth"], format = '%m/%d/%y')
def calc_age(DOB):
    today = date.today()
    age = today.year - DOB.year
    return age 
animals["Age"] = animals["Date of Birth"].apply(calc_age)
animals.drop("Date of Birth", axis = 1, inplace=True)

# Convert MonthYear to datetime then extract month and then year.
animals["MonthYear"] = pd.to_datetime(animals["MonthYear"], format = "%b-%y")
def extract_month(Admit_time):
    return Admit_time.month
def extract_year(Admit_time):
    return Admit_time.year
animals["MonthAdmit"] = animals["MonthYear"].apply(extract_month)
animals["YearAdmit"] = animals["MonthYear"].apply(extract_year)
animals.drop("MonthYear", axis = 1, inplace=True)
             
# Convert Age upon Outcome to days for consistent format. Assume 30 days in a month, 365 days in a year.
def convertdays(OutAge):
    number, category = OutAge.split()
    number = int(number)                  
    if category == "days":
        multiplier = 1
    elif category == "weeks":
        multiplier = 7
    elif category == "months":
        multiplier = 30   
    elif category == "years":
        multiplier = 365  
    else:
        multiplier = 0  
    return multiplier * number
animals["Age upon Outcome (Days)"] = animals["Age upon Outcome"].apply(convertdays)
animals.drop("Age upon Outcome", axis=1, inplace=True)

# Convert DateTime to datetime and extract the time as well as the day since month and year were already extracted
# Code below was generated from Chat GPT 
animals["DateTime"] = pd.to_datetime(animals["DateTime"], format="mixed", errors="coerce", utc=True).dt.tz_convert(None)   

# Now extract date of admit and time since month and year was already extracted from MonthYear
def extract_day(Admit_time):
    return Admit_time.day
def extract_time(Admit_time):
    return Admit_time.time()
animals["DayAdmit"] = animals["DateTime"].apply(extract_day)
animals["TimeAdmit"] = animals["DateTime"].apply(extract_time)
# Convert time into 

animals.drop("DateTime", axis = 1, inplace=True)

In [464]:
animals.info()
animals.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131148 entries, 0 to 131147
Data columns (total 12 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Outcome Type             131108 non-null  object
 1   Outcome Subtype          65802 non-null   object
 2   Animal Type              131148 non-null  object
 3   Sex upon Outcome         131148 non-null  object
 4   Breed                    131148 non-null  object
 5   Color                    131148 non-null  object
 6   Age                      131148 non-null  int64 
 7   MonthAdmit               131148 non-null  int64 
 8   YearAdmit                131148 non-null  int64 
 9   Age upon Outcome (Days)  131148 non-null  int64 
 10  DayAdmit                 131148 non-null  int64 
 11  TimeAdmit                131148 non-null  object
dtypes: int64(5), object(7)
memory usage: 12.0+ MB


,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Breed,Color,Age,MonthAdmit,YearAdmit,Age upon Outcome (Days),DayAdmit,TimeAdmit
0,Transfer,Partner,Bird,Unknown,Duck,Yellow/Black,11,6,2014,14,10,05:00:00
1,Transfer,Partner,Bird,Unknown,Duck,Yellow/Black,11,6,2014,14,10,05:00:00
2,Transfer,Partner,Bird,Unknown,Duck,Yellow/Black,11,6,2014,14,10,05:00:00
3,Transfer,Partner,Bird,Unknown,Duck,Yellow/Black,11,6,2014,14,10,05:00:00
4,Transfer,Partner,Bird,Unknown,Duck,Yellow/Black,11,6,2014,14,10,05:00:00


In [465]:
# Now I will convert everything else through One-hot encoding to make the categorical data numeric while replacing missing values
# Group animals of the same type for replacing missing data with the mode
animals['Outcome Type'] = animals.groupby(['Animal Type'])['Outcome Type'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'Unknown'))
animals['Outcome Type'] = animals['Outcome Type'].astype('category')
animals = pd.get_dummies(animals, columns=['Outcome Type'], drop_first=True)

animals['Outcome Subtype'] = animals.groupby(['Animal Type'])['Outcome Subtype'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'Unknown'))
animals['Outcome Subtype'] = animals['Outcome Subtype'].astype('category')
animals = pd.get_dummies(animals, columns=['Outcome Subtype'], drop_first=True)

animals['Animal Type'] = animals['Animal Type'].astype('category')
animals['Sex upon Outcome'] = animals['Sex upon Outcome'].astype('category')
animals['Breed'] = animals['Breed'].astype('category')
animals['Color'] = animals['Color'].astype('category')

categorical_cols = ['Animal Type', 'Sex upon Outcome', 'Breed', 'Color']
for column in categorical_cols:
    animals = pd.get_dummies(animals, columns=[column], drop_first=True)

In [466]:
animals.info()
animals.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131148 entries, 0 to 131147
Columns: 3146 entries, Age to Color_Yellow/Yellow
dtypes: bool(3140), int64(5), object(1)
memory usage: 398.7+ MB


Age                          0
MonthAdmit                   0
YearAdmit                    0
Age upon Outcome (Days)      0
DayAdmit                     0
                            ..
Color_Yellow/Orange Tabby    0
Color_Yellow/Red             0
Color_Yellow/Tan             0
Color_Yellow/White           0
Color_Yellow/Yellow          0
Length: 3146, dtype: int64